# Курс Введение в обработку естественного языка / Практическое задание урока 5. Part-of-Speech разметка, NER, извлечение отношений

## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации  
2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки  
3. вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)  
4. взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)    
5. сравнить все реализованные методы сделать выводы

### загрузка данных

In [ ]:
!pip install pyconll

In [ ]:
import pyconll

In [ ]:
!mkdir datasets

In [ ]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-07-07 18:04:19--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   106MB/s    in 0.7s    

2021-07-07 18:04:21 (106 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-07-07 18:04:21--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [ ]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [ ]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [ ]:
import nltk

from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [ ]:
def conver_to_test_data(full):
    res = []
    for sent in full:
        sub_res = []
        for token in sent:
            sub_res.append((token.form, token.upos))
        res.append(sub_res)
    return res

In [ ]:
def conver_to_test_sent(full):
    res = []
    for sent in full:
        sub_res = []
        for token in sent:
            sub_res.append(token.form)
        res.append(sub_res)
    return res

In [ ]:
train_data = conver_to_test_data(full_train)

In [ ]:
test_data = conver_to_test_data(full_test)
print(test_data[0:10])

[[('Алгоритм', 'NOUN'), (',', 'PUNCT'), ('от', 'ADP'), ('имени', 'NOUN'), ('учёного', 'NOUN'), ('аль', 'PART'), ('-', 'PUNCT'), ('Хорезми', 'PROPN'), (',', 'PUNCT'), ('-', 'PUNCT'), ('точный', 'ADJ'), ('набор', 'NOUN'), ('инструкций', 'NOUN'), (',', 'PUNCT'), ('описывающих', 'VERB'), ('порядок', 'NOUN'), ('действий', 'NOUN'), ('исполнителя', 'NOUN'), ('для', 'ADP'), ('достижения', 'NOUN'), ('результата', 'NOUN'), ('решения', 'NOUN'), ('задачи', 'NOUN'), ('за', 'ADP'), ('конечное', 'ADJ'), ('время', 'NOUN'), ('.', 'PUNCT')], [('В', 'ADP'), ('старой', 'ADJ'), ('трактовке', 'NOUN'), ('вместо', 'ADP'), ('слова', 'NOUN'), ('"', 'PUNCT'), ('порядок', 'NOUN'), ('"', 'PUNCT'), ('использовалось', 'VERB'), ('слово', 'NOUN'), ('"', 'PUNCT'), ('последовательность', 'NOUN'), ('"', 'PUNCT'), (',', 'PUNCT'), ('но', 'CCONJ'), ('по', 'ADP'), ('мере', 'NOUN'), ('развития', 'NOUN'), ('параллельности', 'NOUN'), ('в', 'ADP'), ('работе', 'NOUN'), ('компьютеров', 'NOUN'), ('слово', 'NOUN'), ('"', 'PUNCT'), (

In [ ]:
test_sent = conver_to_test_sent(full_test)[0]
print(test_sent)

['Алгоритм', ',', 'от', 'имени', 'учёного', 'аль', '-', 'Хорезми', ',', '-', 'точный', 'набор', 'инструкций', ',', 'описывающих', 'порядок', 'действий', 'исполнителя', 'для', 'достижения', 'результата', 'решения', 'задачи', 'за', 'конечное', 'время', '.']


### 1.Проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации

In [ ]:
unigram_tagger = UnigramTagger(train_data)
display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8772537323492737

In [ ]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8829828463586425

In [ ]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.882081353418933

In [ ]:
# Комбинация тэггеров
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data)

0.8814747413473528

### 2.Написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_sent_train = []
for sent in full_train[:]:
    fdata_sent_train.append([token.form.lower() for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form.lower() for token in sent])
    
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [ ]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label) 
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [ ]:
# берем разные векторайзеры, анализируем по буквам
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 149809)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.943644053516665
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9471826239342163
(871526, 149809)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9487749806221144


In [ ]:
# берем разные векторайзеры, анализируем по словам
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='word')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 99485)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.7531847133757962
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.7722255922892866
(871526, 99485)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.7532605398847437


In [ ]:
# В случае совмещения двух признаков (буквы, слова) код успешно отработал только при первом запуске
# При последующих запусках выдавал ошибку. 
# Для ликвидации ошибки делалось уменьшение n_jobs. 
# После запуска в другой день и установки  n_jobs=4 ошибка не выдалась.
import numpy as np
from scipy.sparse import hstack

# берем векторайзер TfidfVectorizer по буквам и HashingVectorizer по словам
scaler = StandardScaler(with_mean=False)
coder_1 = TfidfVectorizer(ngram_range=(1, 5), analyzer='char')
coder_2 = HashingVectorizer(ngram_range=(1, 5), analyzer='word')

X_train_1 = coder_1.fit_transform(train_tok)
X_test_1 = coder_1.transform(test_tok)

X_train_2 = coder_2.fit_transform(train_tok)
X_test_2 = coder_2.transform(test_tok)


X_train = hstack((X_train_1,X_train_2))
X_test = hstack((X_test_1,X_test_2))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)    


print(X_train.shape)
lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=4)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

print('TfidfVectorizer_char + HashingVectorizer_word :', accuracy_score(test_enc_labels, pred))

(871526, 1198385)
TfidfVectorizer_char + HashingVectorizer_word : 0.9441327132409935


### 3.Вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)

In [ ]:
import gensim

#### word2vec

In [ ]:
train_word2vec = fdata_sent_train
train_word2vec.extend(fdata_sent_test)

In [ ]:
w2v_model = gensim.models.Word2Vec(min_count=1, negative=20, sg=1, window=2)
w2v_model.build_vocab(train_word2vec)
w2v_model.train(train_word2vec, epochs=5, total_examples=w2v_model.corpus_count)

(3850208, 4951090)

In [ ]:
X_train_w2v = [w2v_model.wv[word] for word in train_tok]
X_test_w2v = [w2v_model.wv[word] for word in test_tok]

In [ ]:
lr = LogisticRegression(random_state=0, max_iter=35) # max_iter изменено с 25 на 35
lr.fit(X_train_w2v, train_enc_labels)
pred = lr.predict(X_test_w2v)
accuracy_score(test_enc_labels, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7469837899774205

#### fasttext

In [ ]:
ft_model = gensim.models.FastText(train_word2vec, min_count=1, negative=20, sg=1, window=5)

In [ ]:
X_train_ft = [ft_model.wv[word] for word in train_tok]
X_test_ft = [ft_model.wv[word] for word in test_tok]

In [ ]:
lr = LogisticRegression(random_state=0, max_iter=35) # max_iter изменено с 25 на 35
lr.fit(X_train_ft, train_enc_labels)
pred = lr.predict(X_test_ft)
accuracy_score(test_enc_labels, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8727631179860479

#### 4.взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
lr = LogisticRegression(random_state=0, max_iter=25)

In [ ]:
windows = [1,2,3,4,5,6,7,8,9]
for i in windows:
    w2v_model = gensim.models.Word2Vec(min_count=1, negative=20, sg=1, window=i)
    w2v_model.build_vocab(train_word2vec)
    w2v_model.train(train_word2vec, epochs=5, total_examples=w2v_model.corpus_count)
    X_train_w2v = [w2v_model.wv[word] for word in train_tok]
    X_test_w2v = [w2v_model.wv[word] for word in test_tok]
    lr.fit(X_train_w2v, train_enc_labels)
    pred = lr.predict(X_test_w2v)
    print('word2vec, window=', i, accuracy_score(test_enc_labels, pred))

word2vec, window= 1 0.7483992181444411
word2vec, window= 2 0.7394432649209719
word2vec, window= 3 0.7370505173052944
word2vec, window= 4 0.7279766117345735
word2vec, window= 5 0.7279681865669126
word2vec, window= 6 0.7235449735449735
word2vec, window= 7 0.7155242139318573
word2vec, window= 8 0.7166868870690527
word2vec, window= 9 0.7158696458059516


In [ ]:
windows = [1,2,3,4,5,6,7,8,9]
for i in windows:
    ft_model = gensim.models.FastText(train_word2vec, min_count=1, negative=20, sg=1, window=i)
    X_train_ft = [ft_model.wv[word] for word in train_tok]
    X_test_ft = [ft_model.wv[word] for word in test_tok]
    lr.fit(X_train_ft, train_enc_labels)
    pred = lr.predict(X_test_ft)
    print('fasttext, window=', i, accuracy_score(test_enc_labels, pred))

fasttext, window= 1 0.8582886799447309
fasttext, window= 2 0.864000943618778
fasttext, window= 3 0.8595440299261955
fasttext, window= 4 0.8590553702018671
fasttext, window= 5 0.8587015131601119
fasttext, window= 6 0.851380042462845
fasttext, window= 7 0.8503100461699188
fasttext, window= 8 0.8453476224176861
fasttext, window= 9 0.8446651838371584


#### 5.сравнить все реализованные методы сделать выводы

* 0.88147 - скор по комбинации из UnigramTagger BigramTagger TrigramTagger при методе nltk.tag
* 0.94877 - скор LogisticRegression в сочетании с TfidfVectorizer по буквам при методе Vectorizer
* 0.86400 - скор LogisticRegression в сочетании с  FastText по window= 2 при методе эмбедингов 

**Вывод:**
* для рассматриваемого корпуса наилучшим является исследование по буквам, а не по словам 





## Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера не забудьте разделить выборку

In [2]:
!pip install corus

     |████████████████████████████████| 92kB 3.8MB/s 


In [3]:
import corus

In [4]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-07-17 18:56:02--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   226KB/s    in 9.5s    

2021-07-17 18:56:13 (196 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [5]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [6]:
!rm collection5.zip

In [7]:
!ls

Collection5  sample_data


In [8]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
rec = next(records)

In [9]:
rec.spans

[Ne5Span(
     index='T1',
     type='ORG',
     start=0,
     stop=12,
     text='Рособрнадзор'
 ), Ne5Span(
     index='T2',
     type='ORG',
     start=43,
     stop=55,
     text='Рособрнадзор'
 ), Ne5Span(
     index='T3',
     type='GEOPOLIT',
     start=104,
     stop=110,
     text='России'
 ), Ne5Span(
     index='T4',
     type='PER',
     start=111,
     stop=127,
     text='Дмитрий Медведев'
 ), Ne5Span(
     index='T5',
     type='ORG',
     start=151,
     stop=208,
     text='Федеральной службы по надзору в сфере образования и науки'
 ), Ne5Span(
     index='T6',
     type='PER',
     start=241,
     stop=256,
     text='Ивана Муравьева'
 ), Ne5Span(
     index='T7',
     type='PER',
     start=377,
     stop=391,
     text='Любовь Глебова'
 ), Ne5Span(
     index='T8',
     type='ORG',
     start=405,
     stop=419,
     text='Рособрнадзором'
 ), Ne5Span(
     index='T9',
     type='LOC',
     start=506,
     stop=524,
     text='Пензенскую область'
 ), Ne5Span(
     in

In [10]:
!pip install razdel

In [11]:
from razdel import tokenize

In [12]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        is_person = False
        for person in rec.spans:
            if (token.start >= person.start) and (token.stop <= person.stop):
                is_person = True
                break
        words.append([token.text, 'PERSON' if is_person else 'NO_PERSON'])
    words_docs.extend(words)

In [13]:
import pandas as pd

In [14]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [15]:
df_words['tag'].value_counts()

NO_PERSON    219099
PERSON        46221
Name: tag, dtype: int64

In [16]:
df_words.head(20)

,word,tag
0,Р,PERSON
1,.,PERSON
2,Абрамович,PERSON
3,переизбран,NO_PERSON
4,на,NO_PERSON
5,пост,NO_PERSON
6,председателя,NO_PERSON
7,Думы,PERSON
8,Чукотки,PERSON
9,.,NO_PERSON


### 1.взять нер из nltk

In [1]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk

In [17]:
nltk.download('maxent_ne_chunker')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [18]:
nltk.pos_tag(nltk.word_tokenize(rec.text))

[('Е.Примаков', 'JJ'),
 ('назначен', 'NNP'),
 ('главным', 'NNP'),
 ('по', 'NNP'),
 ('ГЛОНАССу', 'NNP'),
 ('Председателем', 'NNP'),
 ('обновленного', 'NNP'),
 ('совета', 'NNP'),
 ('директоров', 'NNP'),
 ('федерального', 'NNP'),
 ('сетевого', 'NNP'),
 ('оператора', 'NNP'),
 ('в', 'NNP'),
 ('сфере', 'NNP'),
 ('навигационной', 'NNP'),
 ('деятельности', 'NNP'),
 ('``', '``'),
 ('НИС', 'FW'),
 ('ГЛОНАСС', 'NN'),
 ("''", "''"),
 ('избран', 'JJ'),
 ('экс-глава', 'JJ'),
 ('Торгово-промышленной', 'JJ'),
 ('палаты', 'NN'),
 ('РФ', 'NNP'),
 ('Евгений', 'NNP'),
 ('Примаков', 'NNP'),
 ('.', '.'),
 ('Как', 'VB'),
 ('сообщили', 'JJ'),
 ('в', 'NNP'),
 ('пресс-службе', 'NN'),
 ('компании', 'NN'),
 (',', ','),
 ('на', 'NNP'),
 ('этом', 'NNP'),
 ('посту', 'NNP'),
 ('Е.Примаков', 'NNP'),
 ('сменит', 'NNP'),
 ('министра', 'NNP'),
 ('по', 'NNP'),
 ('делам', 'NNP'),
 ('гражданской', 'NNP'),
 ('обороны', 'NNP'),
 (',', ','),
 ('чрезвычайным', 'NNP'),
 ('ситуациям', 'NNP'),
 ('и', 'NNP'),
 ('ликвидации', 'NNP')

In [19]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(rec.text))) if hasattr(chunk, 'label') }

{('Navteq Яниса Моисидиса', 'PERSON'),
 ('Ассоциации', 'ORGANIZATION'),
 ('ГЛОНАСС', 'ORGANIZATION'),
 ('Глобальная', 'PERSON'),
 ('Дмитрия Медведева', 'PERSON'),
 ('Евгений Примаков', 'PERSON'),
 ('Евгения Москвичева', 'PERSON'),
 ('Евгения Шмелева', 'PERSON'),
 ('МЧС', 'ORGANIZATION'),
 ('Сергея Шойгу', 'PERSON')}

#### Вывод по nltk
NER из nltk классифицирует с ошибками именованные сущности в тексте на русском языке.

### 3.написать свой нер, попробовать разные подходы (с доп информацией и без), также с учётом соседей и без них

In [29]:
records = load_ne5(dir)
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [26]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [27]:
df_words['tag'].value_counts()

None        219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [37]:
df_words.head(20)

,word,tag
0,Рособрнадзор,ORG
1,возглавил,None
2,30-летний,None
3,чиновник,None
4,Рособрнадзор,ORG
5,возглавил,None
6,30-летний,None
7,чиновник,None
8,Премьер-министр,None
9,России,GEOPOLIT


In [30]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [31]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [32]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [33]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [34]:
def custom_standardization(input_data):
        return input_data

def data_prep(train_data, seq_len=1, vocab_size = 30000):    
    
    vocab_size = 30000
    #seq_len = 1

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [35]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gPool = tf.keras.layers.GlobalMaxPooling1D()
        self.fc1 = tf.keras.layers.Dense(300, activation='relu')
        self.fc2 = tf.keras.layers.Dense(50, activation='relu')
        self.fc3 = tf.keras.layers.Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [36]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 223s 18ms/step - loss: 0.2822 - accuracy: 0.9181 - val_loss: 0.2037 - val_accuracy: 0.9394
Epoch 2/5
12444/12444 [==============================] - 221s 18ms/step - loss: 0.1228 - accuracy: 0.9633 - val_loss: 0.2126 - val_accuracy: 0.9417
Epoch 3/5
12444/12444 [==============================] - 219s 18ms/step - loss: 0.1085 - accuracy: 0.9659 - val_loss: 0.2109 - val_accuracy: 0.9416
Epoch 4/5
12444/12444 [==============================] - 226s 18ms/step - loss: 0.1033 - accuracy: 0.9668 - val_loss: 0.2246 - val_accuracy: 0.9415
Epoch 5/5
12444/12444 [==============================] - 224s 18ms/step - loss: 0.1003 - accuracy: 0.9674 - val_loss: 0.2413 - val_accuracy: 0.9412


In [38]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 3, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 229s 18ms/step - loss: 0.2948 - accuracy: 0.9142 - val_loss: 0.2059 - val_accuracy: 0.9384
Epoch 2/5
12444/12444 [==============================] - 230s 18ms/step - loss: 0.1272 - accuracy: 0.9621 - val_loss: 0.2134 - val_accuracy: 0.9416
Epoch 3/5
12444/12444 [==============================] - 229s 18ms/step - loss: 0.1105 - accuracy: 0.9656 - val_loss: 0.2104 - val_accuracy: 0.9423
Epoch 4/5
12444/12444 [==============================] - 229s 18ms/step - loss: 0.1046 - accuracy: 0.9666 - val_loss: 0.2268 - val_accuracy: 0.9421
Epoch 5/5
12444/12444 [==============================] - 227s 18ms/step - loss: 0.1014 - accuracy: 0.9671 - val_loss: 0.2215 - val_accuracy: 0.9408


In [40]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 5, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 224s 18ms/step - loss: 0.2962 - accuracy: 0.9133 - val_loss: 0.2073 - val_accuracy: 0.9380
Epoch 2/5
12444/12444 [==============================] - 225s 18ms/step - loss: 0.1259 - accuracy: 0.9624 - val_loss: 0.2624 - val_accuracy: 0.8940
Epoch 3/5
12444/12444 [==============================] - 225s 18ms/step - loss: 0.1088 - accuracy: 0.9656 - val_loss: 0.2780 - val_accuracy: 0.8933
Epoch 4/5
12444/12444 [==============================] - 224s 18ms/step - loss: 0.1037 - accuracy: 0.9665 - val_loss: 0.2877 - val_accuracy: 0.8928
Epoch 5/5
12444/12444 [==============================] - 225s 18ms/step - loss: 0.1010 - accuracy: 0.9671 - val_loss: 0.3056 - val_accuracy: 0.8931


#### Вывод по modelNER
В собственном нер подходы с учетом соседей и без мало различаются по результатам.

### 2.проверить deeppavlov

In [ ]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 245kB 4.2MB/s 
     |████████████████████████████████| 2.5MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 27.8MB/s 
     |████████████████████████████████| 901kB 42.6MB/s 
     |████████████████████████████████| 911kB 5.3MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 655kB 28.9MB/s 
     |████████████████████████████████| 7.3MB 15.5MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 10.4MB 20.3MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 2.9MB 35.6MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 860kB 36.3MB/s 
     |████████████████████████████████| 6

In [ ]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

2021-07-17 16:03:06.858 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-ned3x7qs
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-ned3x7qs
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23593 sha256=d45f3ee7c7f29ff670f8b4f51d440f5a104fb58b7b0884fa106be2ce89aadb58
  Stored in directory: /tmp/pip-ephem-wheel-cache-m088owbf/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp
     |████████████████████████████████| 110.5MB 41kB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 512kB 35.8MB/s 
     |████████████████████████████████| 3.8MB 34.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha2

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
!python -m deeppavlov install ner_ontonotes_bert_mult

2021-07-17 17:18:22.370 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ner_ontonotes_bert_mult' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/ner/ner_ontonotes_bert_mult.json'
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-z5rg1dck
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-z5rg1dck
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23593 sha256=2b8c009ac6541326f74ba5560210bdd150d44d84f0e88c7ee10271a0ef16a306
  Stored in directory: /tmp/pip-ephem-wheel-cache-b7__et7q/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [ ]:
ner_dp_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
result_1 = ner_dp_model([rus_document])

for i in range(len(result_1[0][0])):
     if result_1 [1][0][i] != 'O':
         print(result_1[0][0][i], result_1[1][0][i])

2021-07-17 17:21:17.966 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz to /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz
100%|██████████| 1.32G/1.32G [05:33<00:00, 3.96MB/s]
2021-07-17 17:26:53.531 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz archive into /root/.deeppavlov/models
2021-07-17 17:27:33.156 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2021-07-17 17:27:33.600 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API



2021-07-17 17:28:08.853 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model
Нью B-GPE
- I-GPE
Йорк I-GPE
США B-GPE
30 B-DATE
апреля I-DATE
2020 I-DATE
01 B-TIME
: I-TIME
01 I-TIME
США B-GPE
Дональда B-PERSON
Трампа I-PERSON
100 B-MONEY
млн I-MONEY
доз I-MONEY
конца B-DATE
2020 I-DATE
года I-DATE
Bloomberg B-ORG


#### Вывод по deeppavlov
В DeepPavlov мультиязычный NER версии ner_ontonotes_bert_mult отлично классифицирует слова русского текста по типу именованных сущностей, к которым они принадлежат (имена собственные, географические названия, даты, валюты и другие).

### 4.Дополнительно: NER в STANZA

In [ ]:
!pip install stanza

     |████████████████████████████████| 337kB 4.2MB/s 


In [ ]:
import stanza
stanza.download('ru')
def stanza_nlp_ru(text):
  nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')
  doc = nlp(text)
  print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
stanza_nlp_ru(rus_document)

2021-07-17 17:53:27 INFO: Downloading default packages for language: ru (Russian)...


2021-07-17 17:55:23 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-07-17 17:55:23 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2021-07-17 17:55:23 INFO: Use device: cpu
2021-07-17 17:55:23 INFO: Loading: tokenize
2021-07-17 17:55:23 INFO: Loading: ner
2021-07-17 17:55:25 INFO: Done loading processors!


entity: Нью-Йорк	type: LOC
entity: США	type: LOC
entity: США	type: LOC
entity: Дональда Трампа	type: PER
entity: Bloomberg	type: ORG


#### Вывод по STANZA
Модель с NER в STANZA отработала быстрее, чем NER в DeepPavlov. Результат для текста на русском языке - отличный.